# Advent of Code

### Setup

In [1]:
### Imports go here
import re
import pandas as pd

## Day 1

### Puzzle 1

For each line, combine the first and last digit in the line (could be the same one) to create a 2 digit number. Add up all numbers to receive the result.

In [8]:
file = open('data/day_1_input.txt', 'r')
codewords = file.read().split()
file.close()

In [19]:
result = 0
for line in codewords:
    digits = re.sub('\D', '', line)
    result += 10 * int(digits[0])
    result += int(digits[-1])

print(result)

54968


### Puzzle 2

Include spelled out values ('one', 'two' etc.) in the calculations from puzzle 1

In [33]:
numbers = {
    'one':'1',
    'two':'2',
    'three':'3',
    'four':'4',
    'five':'5',
    'six':'6',
    'seven':'7',
    'eight':'8',
    'nine':'9',
    '1':'1',
    '2':'2',
    '3':'3',
    '4':'4',
    '5':'5',
    '6':'6',
    '7':'7',
    '8':'8',
    '9':'9'
}

In [35]:
result = 0

for line in codewords:
    digits = ''
    for i in range(0,len(line)):
        linepart = line[i:]
        for key in numbers.keys():
            if linepart.startswith(key):
                digits = digits + numbers[key]
    result += 10 * int(digits[0])
    result += int(digits[-1])
    
print(result)

54094


## Day 2

### Puzzle 1

- Each game, a random number of red, green and blue cubes are placed in a bag.
- Several times, a random number of cubes is drawn and shown to the user (separated by semicolons)

##### Task:

Determine which game results are possible with a bag that contains 12 red, 13 green and 14 blue cubes, and sum up the game IDs

In [4]:
file = open('data/day_2_input.txt', 'r')
cubegames = file.read().splitlines()
file.close()

In [18]:
result = 0

# Parse lines
for line in cubegames:
    gameID = int(line.split(':')[0][5:])
    green = True
    red = True
    blue = True
    
    rounds = line.split(':')[1].split(';')
    
    # go through each round of the game
    for round in rounds:
        cubenumbers = round.strip().split(', ')
        for cube in cubenumbers:
            number = int(cube.split()[0])
            color = cube.split()[1]

            # check each color
            if color == 'red':
                if number > 12:
                    red = False
            elif color == 'green':
                if number > 13:
                    green = False
            else:
                if number > 14:
                    blue = False
    
    # check if conditions were met
    if green and red and blue:
        result += gameID

print(result)

    

2256


### Puzzle 2

For each game, find the minimum number of cubes that needed to be in the bag for the game to be possible. 

Multiply the number for the three colors, and find the sum for all games

In [23]:
result = 0

# Parse lines
for line in cubegames:
    green = 0
    red = 0
    blue = 0
    
    rounds = line.split(':')[1].split(';')

    # go through each round of the game
    for round in rounds:
        cubenumbers = round.strip().split(', ')
        for cube in cubenumbers:
            number = int(cube.split()[0])
            color = cube.split()[1]

            # check each color
            if color == 'red':
                red = max(number, red)
            elif color == 'green':
                green = max(number, green)
            else:
                blue = max(number, blue)
    
    # multiply and add results
    result += (red * green * blue)

print(result)

74229


## Day 3

### Puzzle 1

The input file contains a "matrix" of numbers, symbols and periods. Only numbers that are adjacent, above or below (including diagonally) to a symbol are valid.

##### Task:
Find the sum of all valid numbers

In [26]:
file = open('data/day_3_input.txt', 'r')
engine = file.read().splitlines()
file.close()

In [65]:
symbols = []
numbers = []
gears = []

for y, line in enumerate(engine):
    x = 0
    while x < len(line):
        if line[x] == '.':
            x += 1
        elif line[x].isnumeric():
            if line[x + 1].isnumeric():
                if line[x + 2].isnumeric():
                    numbers.append([int(line[x:x+3]), y, x, 3])
                    x += 3
                else: 
                    numbers.append([int(line[x:x+2]), y, x, 2])
                    x += 2
            else: 
                numbers.append([int(line[x]), y, x, 1])
                x += 1
        elif line[x] == '*':
            gears.append((y,x))
            symbols.append((y,x))
            x += 1
        else:
            symbols.append((y,x))
            x += 1

In [62]:
#check surroundings for symbols based on coordinates and length
def check_surrounding(y,x,l):
    xmin = max(0,x - 1)
    xmax = x + l
    ymin = max(0,y - 1)
    ymax = min(len(engine), y + 1)

    symbol_found = False

    # iterate over all possibilities
    for x_val in range(xmin, xmax + 1):
        for y_val in range(ymin, ymax + 1):
            if (y_val, x_val) in symbols:
                symbol_found = True

    return symbol_found

In [66]:
result = 0

for n in numbers:
        if check_surrounding(n[1],n[2],n[3]):
                result += n[0]

print(result)

539433


### Puzzle 2

A gear is a * symbol that connects to exactly two numbers. The gear ratio is the product of both numbers.

##### Task

Calculate the sum of all gear ratios

In [78]:
# A new version of the check_surrounding() function is necessary

connections = {}

def check_gears(n):
    xmin = max(0,n[2] - 1)
    xmax = n[2] + n[3]
    ymin = max(0,n[1] - 1)
    ymax = min(len(engine), n[1] + 1)

    # iterate over all possibilities
    for x_val in range(xmin, xmax + 1):
        for y_val in range(ymin, ymax + 1):
            if (y_val, x_val) in gears:
                i = gears.index((y_val, x_val))
                parts = connections.get(i, [])
                parts.append(n[0])
                connections[i] = parts

for n in numbers:
    check_gears(n)

In [82]:
result = 0

# Only use gears with exactly two connections

for gear in connections.values():
    if len(gear) == 2:
        ratio = gear[0] * gear[1]
        result += ratio

print(result)

75847567


## Day 5

### Puzzle 1

Each card has numbers before and after the '|' symbol. Find how many numbers appear on both sides. Card value is 2**(n-1) where n is the amount of matching numbers


In [1]:
file = open('data/day_4_input.txt', 'r')
cards = file.read().splitlines()
file.close()

In [21]:
result = 0
winning_numbers = []

for card in cards:
    numbers = card.split(': ')[1].split(' | ')
    left = set(numbers[0].split())
    right = set(numbers[1].split())

    winning_nrs = len(left.intersection(right))
    winning_numbers.append(winning_nrs)
    if winning_nrs > 0:
        points = 2**(winning_nrs - 1)
        result += points
        

result


26443

### Puzzle 2

In [23]:
def recursive_cards(i):
    result = 1
    
    if winning_numbers[i] > 0:
        for j in range(i + 1, i + winning_numbers[i] + 1):
            result += recursive_cards(j)
    
    return result

In [25]:
result = 0
for i in range (0, len(winning_numbers)):
    result += recursive_cards(i)

result

6284877

## Day 5

### Puzzle 1

The input contains mappings in the format "a b c" where a is the start of the source range, b is the start of the target range, and c is the range, meaning `[a:a+c]` get mapped to `[b:b+c]`

##### Task

Map the seed numbers through all the maps and find the lowest resulting number

In [3]:
file = open('data/day_5_input.txt', 'r')
almanach = file.read().strip().split('\n\n')
file.close()

 ##### Parse input

In [4]:
seeds = [int(x) for x in almanach[0].split(' ')[1:]]

maps = []
for page in almanach[1:]:
    map_raw = page.split('\n')[1:]

    this_map = []
    for mapping in map_raw:

        # I'm reordering the map because it makes more sense in my head and makes sorting by source easier
        map = [int(x) for x in  mapping.split()]
        this_map.append([map[1],map[0],map[2]])
    
    this_map.sort()
    maps.append(this_map)

##### Create mapping function

In [3]:
def map_digit(digit, map):
    
    i = 0
    while (i + 1 < len(map)) and (map[i+1][0] <= digit):
        i += 1

    if (map[i][0] <= digit) and (map[i][0] + map[i][2] > digit):
        return digit - map[i][0] + map[i][1]
    else:
        return digit

In [4]:
locations = []

for seed in seeds:
    digit = seed
    for map in maps:
        digit = map_digit(digit, map)

    locations.append(digit)

print(min(locations))

218513636


### Puzzle 2

Treat the seed numbers instead as pairs of initial numbers and a range.

##### Task:

Find the smallest location number based on the new seed numbers

In [18]:
# going through all ranges one by one will take too long, but maybe we can take shortcuts
# instead of calculating the mapping of a single digit, let's try calculating the mapping of a range

def map_range(range, map):
    
    new_ranges = []

    # setup variables
    range_start = range[0]
    remaining_range = range[1]

    # start a loop
    # find right mapping in map
    i = 0

    while(True):
        while (i + 1 < len(map)) and (map[i+1][0] <= range_start):
            i += 1

        # if range start falls within range in the mapping
        if (map[i][0] <= range_start) and (map[i][0] + map[i][2] > range_start):

            # see how far the mapping range goes
            # if it is shorter than the remaining range of the digit:
            if map[i][0] + map[i][2] < range_start + remaining_range:
                
                overlap_length = map[i][0] + map[i][2] - range_start

                # add overlapping range to new_ranges 
                new_ranges.append(range_start - map[i][0] + map[i][1])
                new_ranges.append(overlap_length)

                # update range_start and remaining_range
                range_start = range_start + overlap_length 
                remaining_range = remaining_range - overlap_length

            # if it is not shorter we are done
            else:
                new_ranges.append(range_start - map[i][0] + map[i][1])
                new_ranges.append(remaining_range)
                return new_ranges
            
        # if range start does not fall within range in the mapping
        # check if there is overlap with the next mapping
        elif (i + 1 < len(map)) and (map[i+1][0] < range_start + remaining_range):

            non_overlap_length = map[i+ 1][0] - range_start

            # add non-overlapping range to new_ranges
            new_ranges.append(range_start)
            new_ranges.append(non_overlap_length)

            # update range_start and remaining_range
            range_start = range_start + non_overlap_length 
            remaining_range = remaining_range - non_overlap_length

        # if there is no overlap with the next mapping we are done
        else:
            new_ranges.append(range_start)
            new_ranges.append(remaining_range)
            return new_ranges

In [21]:
current_ranges = seeds
for map in maps:
    new_ranges = []
    for i in range(0,len(current_ranges),2):
        new_ranges.extend(map_range(current_ranges[i:i+2], map))
    current_ranges = new_ranges
    new_ranges = []

print(min(current_ranges[::2]))

81956384
